In [1]:
import numpy as np
import statsmodels.api as sm

def recover_S_ols_summary2(v1m, v2m, step=[0.1,0.05]):
    """
    step:
        - scalar: same step size for both directions
        - list/tuple: [dx, dy]
    """

    # --- handle step input ---
    if isinstance(step, (list, tuple, np.ndarray)):
        dx, dy = step
    else:
        dx = dy = step

    Nx, Ny = v1m.shape
    N = Nx * Ny
    eqs = (Nx - 1) * Ny + Nx * (Ny - 1)

    A = np.zeros((eqs, N))
    b = np.zeros(eqs)

    def idx(i, j):
        return i * Ny + j

    k = 0

    # --- x-direction (using v2m and dx) ---
    for i in range(Nx - 1):
        for j in range(Ny):
            A[k, idx(i + 1, j)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v2m[i, j] + v2m[i + 1, j]) * dx
            k += 1

    # --- y-direction (using v1m and dy) ---
    for i in range(Nx):
        for j in range(Ny - 1):
            A[k, idx(i, j + 1)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v1m[i, j] + v1m[i, j + 1]) * dy
            k += 1

    # remove gauge freedom (fix S[0,0] = 0)
    A = A[:, 1:]

    model = sm.OLS(b, A)
    results = model.fit()

    s_sub = results.params
    s = np.concatenate(([0], s_sub))
    S = s.reshape(Nx, Ny)

    # --- diagnostics ---
    residuals = results.resid
    rss = np.sum(residuals ** 2)
    tss = np.sum(b ** 2)
    ratio = rss / tss if tss > 0 else np.nan

    print(f"RSS:   {rss:.2e}")
    print(f"TSS:   {tss:.2e}")
    print(f"Ratio: {ratio:.2e}")



    return S, results, rss, tss, ratio


In [2]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

def recover_S_ols_summary3(v1m, v2m, step=[0.1, 0.05], alpha=0.05):
    """
    Recover scalar potential S via OLS and report statistical significance.

    Parameters
    ----------
    v1m, v2m : 2D np.ndarray
        Vector field components
    step : float or (dx, dy)
        Grid spacing
    alpha : float
        Significance level for hypothesis tests
    """

    # --- handle step input ---
    if isinstance(step, (list, tuple, np.ndarray)):
        dx, dy = step
    else:
        dx = dy = step

    Nx, Ny = v1m.shape
    N = Nx * Ny
    eqs = (Nx - 1) * Ny + Nx * (Ny - 1)

    A = np.zeros((eqs, N))
    b = np.zeros(eqs)

    def idx(i, j):
        return i * Ny + j

    k = 0

    # --- x-direction ---
    for i in range(Nx - 1):
        for j in range(Ny):
            A[k, idx(i + 1, j)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v2m[i, j] + v2m[i + 1, j]) * dx
            k += 1

    # --- y-direction ---
    for i in range(Nx):
        for j in range(Ny - 1):
            A[k, idx(i, j + 1)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v1m[i, j] + v1m[i, j + 1]) * dy
            k += 1

    # --- remove gauge freedom ---
    A = A[:, 1:]

    model = sm.OLS(b, A)
    results = model.fit()

    # --- recover S ---
    s_sub = results.params
    s = np.concatenate(([0], s_sub))
    S = s.reshape(Nx, Ny)

    # --- diagnostics ---
    residuals = results.resid
    rss = np.sum(residuals ** 2)
    tss = np.sum(b ** 2)
    ratio = rss / tss if tss > 0 else np.nan

    print("\n=== GLOBAL MODEL DIAGNOSTICS ===")
    print(f"Observations:                {len(b)}")
    print(f"Parameters estimated:        {A.shape[1]}")
    print(f"R-squared:                   {results.rsquared:.6f}")
    print(f"Adj. R-squared:              {results.rsquared_adj:.6f}")
    print(f"F-statistic:                 {results.fvalue:.4f}")
    print(f"Prob (F-statistic):          {results.f_pvalue:.4e}")

    print("\n=== CIRCULATION DIAGNOSTIC ===")
    print(f"RSS (residual sum of squares): {rss:.6e}")
    print(f"TSS (total sum of b²):         {tss:.6e}")
    print(f"Circulating / total ratio:     {ratio:.6%}")

    # --- parameter significance ---
    coef_df = pd.DataFrame({
        "coef": results.params,
        "std_err": results.bse,
        "t": results.tvalues,
        "p": results.pvalues
    })

    sig_mask = coef_df["p"] < alpha
    sig_ratio = sig_mask.mean()

    print("\n=== PARAMETER SIGNIFICANCE ===")
    print(f"Significance level α = {alpha}")
    print(f"Significant coefficients: {sig_mask.sum()} / {len(sig_mask)} "
          f"({sig_ratio:.2%})")

    return S, results, rss, tss, ratio


In [3]:
import numpy as np
import statsmodels.api as sm

def recover_S_ols_summary(v1m, v2m, step=0.1):

    Nx, Ny = v1m.shape
    N = Nx * Ny
    eqs = (Nx - 1) * Ny + Nx * (Ny - 1)
    A = np.zeros((eqs, N))
    b = np.zeros(eqs)

    def idx(i, j):
        return i * Ny + j

    k = 0
   
    for i in range(Nx - 1):
        for j in range(Ny):
            A[k, idx(i+1, j)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v2m[i, j] + v2m[i+1, j]) * step
            k += 1

    
    for i in range(Nx):
        for j in range(Ny - 1):
            A[k, idx(i, j+1)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v1m[i, j] + v1m[i, j+1]) * step
            k += 1

   
    A = A[:, 1:]

  
    model = sm.OLS(b, A)
    results = model.fit()

    s_sub = results.params
    s = np.concatenate(([0], s_sub))
    S = s.reshape(Nx, Ny)

 
    residuals = results.resid            
    rss = np.sum(residuals**2)           
    tss = np.sum(b**2)                   
    ratio = rss / tss if tss > 0 else np.nan  

    print(f"RSS (residual sum of squares): {rss:.6e}")
    print(f"TSS (total sum of b²):         {tss:.6e}")
    print(f"Circulating / total ratio:     {ratio:.6%}")

    return S, results, rss, tss, ratio

In [4]:
import numpy as np
import statsmodels.api as sm

def ols_S_summary(S_num, S_theory):
    """
    OLS between two S matrices, using statsmodels.
    Prints full summary.
    """
    # flatten matrices
    y = np.array(S_num, dtype=float).reshape(-1)
    X = np.array(S_theory, dtype=float).reshape(-1)

    # add intercept
    X = sm.add_constant(X)  # column [1, X]

    # fit model
    model = sm.OLS(y, X).fit()

    # print summary
    print(model.summary())

    return model


In [5]:
import numpy as np
import statsmodels.api as sm

def ols_S_summary2(S_num, S_theory):
    """
    OLS between two S matrices, using statsmodels.
    Prints full summary + RSS, TSS, RSS/TSS.
    """

    # flatten matrices
    y = np.array(S_num, dtype=float).reshape(-1)
    X = np.array(S_theory, dtype=float).reshape(-1)

    # add intercept
    X = sm.add_constant(X)  # [1, X]

    # fit model
    model = sm.OLS(y, X).fit()

    # print standard summary
    print(model.summary())

    # ===== extra diagnostics =====
    residuals = model.resid
    y_hat = model.fittedvalues
    y_mean = np.mean(y)

    RSS = np.sum(residuals ** 2)
    TSS = np.sum((y - y_mean) ** 2)
    RSS_TSS_ratio = RSS / TSS

    print("\nAdditional diagnostics:")
    print(f"RSS      = {RSS:.2e}")
    print(f"TSS      = {TSS:.2e}")
    print(f"RSS/TSS  = {RSS_TSS_ratio:.2e}")
    print(f"1 - R^2  = {(1 - model.rsquared):.2e}")


    return model, RSS, TSS, RSS_TSS_ratio


In [6]:
import numpy as np
import statsmodels.api as sm

def recover_S_ols(v1m, v2m, step=1.0, drop_gauge=True):
    """
    Recover scalar potential S from 2D vector field (v1m, v2m)
    using finite differences and OLS.

    Parameters
    ----------
    v1m, v2m : ndarray (Nx, Ny)
        Measured gradient components
    step : float
        Grid spacing
    drop_gauge : bool
        Whether to fix gauge by dropping one variable (recommended)

    Returns
    -------
    S : ndarray (Nx, Ny)
        Recovered entropy surface
    results : statsmodels regression results
    """

    Nx, Ny = v1m.shape
    N = Nx * Ny

    def idx(i, j):
        return i * Ny + j

    eqs = (Nx - 1) * Ny + Nx * (Ny - 1)
    A = np.zeros((eqs, N))
    b = np.zeros(eqs)

    k = 0

    # x-direction differences (use v2)
    for i in range(Nx - 1):
        for j in range(Ny):
            A[k, idx(i+1, j)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v2m[i, j] + v2m[i+1, j]) * step
            k += 1

    # y-direction differences (use v1)
    for i in range(Nx):
        for j in range(Ny - 1):
            A[k, idx(i, j+1)] = 1
            A[k, idx(i, j)] = -1
            b[k] = 0.5 * (v1m[i, j] + v1m[i, j+1]) * step
            k += 1

    # Gauge fixing: remove first column (S_00 = 0)
    if drop_gauge:
        A_reg = A[:, 1:]
    else:
        A_reg = A

    model = sm.OLS(b, A_reg)
    results = model.fit()

    if drop_gauge:
        s = np.concatenate([[0.0], results.params])
    else:
        s = results.params

    S = s.reshape(Nx, Ny)

    return S, results


## calculate theoretical S

In [7]:
import numpy as np

def compute_S_from_gm(gm_matrix):
    gm_array = np.array(gm_matrix)  # shape (n, n, 2)
    g = gm_array[:, :, 0]
    m = gm_array[:, :, 1]
    S = 3 * (np.log(g) + np.log(m))
    S = S - S[0, 0] 
    return S

## plots 

In [8]:
import numpy as np
import matplotlib.pyplot as plt

def Splot(St3, g2_label):
    st_arr = np.array(St3)
    X, Y = np.meshgrid(np.arange(1000, 1000 + 100 * st_arr.shape[1], 100),
                       np.arange(1000, 1000 + 100 * st_arr.shape[0], 100))
    
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, st_arr, cmap='viridis')

    
    ax.set_box_aspect((1, 1, 0.8))
    ax.set_xlabel('G1')
    ax.set_ylabel(g2_label)

    
    ax.text2D(0.05, 0.5, "S ", transform=ax.transAxes, 
              rotation=0, va='center', fontsize=12)

    
    plt.subplots_adjust(left=0.2)
    plt.show()


In [9]:
import numpy as np
import matplotlib.pyplot as plt

def Splot2(St3, g2_label):
    st_arr = np.array(St3)
    n_rows, n_cols = st_arr.shape

    
    x = 1300 - 100 * np.arange(n_cols)
    y = 1300 - 100 * np.arange(n_rows)
    X, Y = np.meshgrid(x, y)

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, st_arr, cmap='viridis')

    
    ax.set_box_aspect((1, 1, 0.8))
    ax.set_xlabel('G1')
    ax.set_ylabel(g2_label)

    
    ax.text2D(
        0.05, 0.5, "S",
        transform=ax.transAxes,
        va='center',
        fontsize=12
    )

    
    
    plt.subplots_adjust(left=0.2)
    plt.show()



In [10]:
import numpy as np

def build_gm_matrix(start=1, end=1.9, step=1):
    grid = np.arange(start, end + step, step)
    G, M = np.meshgrid(grid, grid, indexing="ij")
    gm_matrix = np.stack([G, M], axis=2)  # shape (n, n, 2)
    return gm_matrix

In [11]:
import numpy as np

def compute_Fs(v1, v2, N, eta, alpha, beta=1.0, eps=1e-12):
    """
    F(beta, v1, v2) = N * ( eta*log(beta) + log( (v1 - v2) / (v2^{-alpha} - v1^{-alpha}) ) )

    Parameters
    ----------
    v1, v2 : np.ndarray (n x n)
        Must be positive for the power terms. Also need the ratio to be positive element-wise.
    N, eta, alpha : float
    beta : float, default=1.0
    eps : float, default=1e-12
        Small constant to avoid division by zero and log(0).

    Returns
    -------
    F : np.ndarray (n x n)
    """
    v1 = np.asarray(v1, dtype=float)
    v2 = np.asarray(v2, dtype=float)

    # term eta log beta (scalar, broadcast)
    term1 = eta * np.log(beta)

    # numerator and denominator (element-wise)
    num = v1 - v2
    den = (v2 ** (-alpha)) - (v1 ** (-alpha))

    # safe ratio: avoid zero division; but note sign must be positive for real log
    ratio = num / (den )

    # If you want hard safety, uncomment the next line to prevent log of non-positive values:
    # ratio = np.maximum(ratio, eps)

    F = N * (term1 + np.log(ratio))

    return F



In [12]:
import numpy as np

def compute_Fc(v1, v2, N, eta, alpha, beta=1.0):
    """
    Compute F(beta, v1, v2) element-wise for n x n matrices.

    Parameters
    ----------
    v1 : np.ndarray
        nu_1 matrix (n x n), must be positive
    v2 : np.ndarray
        nu_2 matrix (n x n), must be positive
    N : float
    eta : float
    alpha : float
    beta : float, default=1.0 (kept in formula)

    Returns
    -------
    F : np.ndarray
        n x n matrix
    """

    term1 = eta * np.log(beta)                 # scalar (0 if beta=1)
    term2 = (alpha - 1) * np.log(v1 + v2)      # element-wise
    term3 = np.log(v1)                         # element-wise
    term4 = np.log(v2)                         # element-wise

    F = N * (term1 + term2 + term3 + term4)

    return F


In [13]:
import numpy as np

def normalize_F(F):
    """
    Convert F to -F and shift the whole matrix
    so that F[0, 0] becomes 0.

    Parameters
    ----------
    F : np.ndarray
        Input matrix

    Returns
    -------
    F_norm : np.ndarray
        Normalized matrix
    """
    F_neg = -F
    F_norm = F_neg - F_neg[0, 0]
    return F_norm


In [14]:
import numpy as np

def compute_F(v1, v2, N, eta, alpha1, beta=1.0):
    """
    Compute F(beta, v1, v2) element-wise and return a matrix.

    Parameters
    ----------
    v1 : np.ndarray
        nu_1 matrix
    v2 : np.ndarray
        nu_2 matrix
    N : float
    eta : float
    alpha1 : float
    alpha2 : float
    beta : float, default=1.0

    Returns
    -------
    F : np.ndarray
        Matrix with same shape as v1 and v2
    """
    F = N * (
        
        + alpha1 * np.log(v1)
        + eta * np.log(v2)
    )
    return F

In [17]:
import numpy as np

def matrix_difference_summary(S_num, S_theory):
    """
    Compare two matrices element-wise (no regression).
    Reports difference-based diagnostics: RSS, TSS, RMSE, MAE, etc.
    """

    # flatten matrices
    y = np.array(S_num, dtype=float).reshape(-1)
    y_ref = np.array(S_theory, dtype=float).reshape(-1)

    if y.shape != y_ref.shape:
        raise ValueError("S_num and S_theory must have the same shape")

    # element-wise difference
    diff = y - y_ref

    # diagnostics
    RSS = np.sum(diff ** 2)
    TSS = np.sum((y - np.mean(y)) ** 2)
    RSS_TSS_ratio = RSS / TSS

    RMSE = np.sqrt(np.mean(diff ** 2))
    MAE = np.mean(np.abs(diff))
    max_abs_error = np.max(np.abs(diff))

    print("Matrix difference diagnostics:")
    print(f"RSS      = {RSS:.2e}")
    print(f"TSS      = {TSS:.2e}")
    print(f"RSS/TSS  = {RSS_TSS_ratio:.2e}")
    print(f"RMSE     = {RMSE:.2e}")
    print(f"MAE      = {MAE:.2e}")
    print(f"Max |Δ|  = {max_abs_error:.2e}")

    return {
        # "diff": diff.reshape(np.shape(S_num)),
        "RSS": RSS,
        "TSS": TSS,
        "RSS/TSS": RSS_TSS_ratio,
        "RMSE": RMSE,
        "MAE": MAE,
        "MaxAbsError": max_abs_error
    }